### Código que  por cada columna de datos se intenta predecir su incremento en F dias usando los  incrementos que ha habido de otras columnas  desde hace P dias.  El conjunto de columnas que se usan para predecir es multiplo de p ya que se utilizan todos los pasados posibles de una columna.


#### Antes de ejecutar este código hay que  ejecutar el llamado limpiarCeros pues si en los datos hay algun cero, se prodicira un error  al llevar a cabo la division por 0 y dar de resultado infinito

NO usa el incremento dos a dos

In [ ]:
import csv
import sys
import numpy as np
import math
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from math import sqrt

d = pd.read_csv("raw.csv",index_col=False)


In [ ]:
def pega(sentence):
    sent_str = ""
    for i in sentence:
        sent_str += str(i)
    return sent_str
def cat(outfilename, line):
    with open(outfilename, 'w') as outfile:
            outfile.write(line)
            outfile.close()
def cat2(outfilename, line):
    with open(outfilename,'a') as outfile:
            outfile.write(line)
            outfile.close()

In [ ]:
prueba_numero =1

f = 100
p = 50
prefix ='C:/Users/Celia/Desktop/4º/TFG/codigo/python'
path = pega([prefix,'/data20_10_scaled_inc',prueba_numero,'.txt'])

cat(path,pega(["p: ",p," f: ",f,"\n"]))
errores =pega([prefix,'/rmse',prueba_numero,'.txt'])
cat(errores,pega(["Rmse mejores",prueba_numero,"\n"]))
#archivo registro de pruebas
n=pega([prefix,'/registro_pruebas1.txt'])
cat2(n,pega(['nº',prueba_numero ,' columna_predecir: BDIY_Index_Open ',"p: ",p," f: ",f," columnas: ",len(d.columns)," n: ",3]))


max_col = 3


In [ ]:

for col in list(d.columns.values):
    print(col)
    if(col=='BDIY_Index_Open'):# probar solo con euro
        
        present = d[(p):(len(d)-f)][col].reset_index()
        future = d[(f+p):len(d)][col].reset_index()
        # la etiqueta es el % de incremento en f días (saltando los p+1 primeros)

        label1 = future.subtract(present, fill_value=0).reset_index()
        label2 = (label1[col]/present[col]).reset_index()
        label=label2.drop(label2.columns[[0]], axis='columns')


        array = [0] * len(label)
        zeros = pd.DataFrame(data=array)

        a = label.values

        rmse =sqrt(mean_squared_error(array, a))
        cat2(path,pega([col," ", rmse,"\n"]))
        # hasta aqui columna label y error del naive
        
        rmse_up_to_now = 1
        columns= []
        column_names = []

        n = 0
        puntoFijo = False
        cambios = 0
        # miemtras que no se lleven el máximo de columnas y siga mejorando
        while n < max_col and (not puntoFijo):
            print(pega(["n=", n]))
            ndf = columns
            ini = 0
            fin = len(d.columns)
            
            #recorre todas las columnas
            for i in np.arange(ini, fin):
                print(i)
                #nombre de la columna
                nombres = pega([list(d.columns.values)[i],1])
                #si esa columna no ha sido cogida ya antes
               
                if not (nombres in column_names):
                    pred = i
                    s = np.arange(1, len(d)+1)# filas de la tabla
                    vs = np.arange(0, p-1) # pasados a utilizar es ahasta p-1 porque no incluyepe la funcion arange
                    name = list(d.columns.values)[pred]
                    names = []
                    #para la columna  i con la que quiero hacer la prediccion  en names estan los posibles pasados de esa columna a usar
                    for v in vs:
                        names.append(pega([name, (p-v),"-"]))
                    df=pd.DataFrame(columns=names)
                    # desde el dia p+1(cosas de indices) hasta el final menos futuro --> tabla
                    for i in np.arange(p,len(d)-f):
                        #para cada filla nrwo representa la fila actual
                        nrow=[]
                        #rellenar las columnas de (EuroPasado10, EuroPasado9, EuroPadaso8....)
                        for v in vs:
                            #calcular los incrementos que ha habido  en un marco temporal de 10 dias atras
                            #d[ el dia a predeccir - (p-v= rango de 10 a 1) dias atras]
                            nrow.append(((d.iloc[i-(p-v)][pred])-(d.iloc[i-1][pred]))/ (d.iloc[i-1][pred]))
                        df.append(nrow)
                        df = df.append(pd.Series(nrow, index=df.columns ), ignore_index=True)
                     
                    
                    
                    # se añade el mejor conjunto de columnas de la iteracion anterior de n si no es vacio
                    if(len(columns)!=0):
                        df=pd.concat([columns, df], axis=1, sort=False)
                    
                    # crear la matriz de datos con label
                    datos = pd.concat([label, df], axis=1, sort=False)
                    
                   
                    
                    
                    #-----------------EVALUACION---------------------------------------
                   
                        
                            
                   
                    
                    trainsize=1500 # tamaño de inicion del conjunto de entrenamiento
                    s =np.arange(trainsize,len(datos)-f)
                   
                    predichos = []
                    reales=[]
                    for j in s:
                        train = datos[1:j][:]
                        lr = linear_model.LinearRegression()
                        lr.fit(train.iloc[:,1:], train.iloc[:,0:1])
                        
                        test=datos.iloc[[j+f],1:]
                        Y_pred = lr.predict(test)
                        real =datos.iloc[[j+f],0:1]
                        
                        valor_p =Y_pred[0][0]
                        predichos.append(valor_p)
                        valor_r=real.iloc[0,0]
                        reales.append(valor_r)
                    
                    
                    rmse =sqrt(mean_squared_error(reales, predichos))
                    cambios +=1
                    
                    #si mejora
                    if(not(np.isnan(rmse)) and rmse<rmse_up_to_now):
                        rmse_up_to_now=rmse
                        ndf= df
                        cat2(errores,pega([col, " con ", name," iteracion", n," ", rmse_up_to_now,"\n"]))
                
                        
             
            print(pega(["mejores columnas iteracion: ",n]))
            print(ndf.columns.values)
                  
            # aqui termina la busqueda de la combinacion de pasasdos para las columnas i para predecir la columna col         
            if(cambios>0 or len(columns==0) or not (columns.columns.values ==ndf.columns.values) ) :
               
                columns = ndf
              
                columns_names = columns.columns.values
                
                print("cambio columnas")
                print(column_names)
                n+=1
            else:
                print("vuelta sin cabios")
                puntoFijo = True
                    
        print("Acabado")
        print(rmse_up_to_now)
        print(ndf.columns.values)
    
                    
               
                    
               
                    
                   
                    
                    
                    
                    
          

   

    
    
     


    
    
   
    
    
    


In [ ]:
print(d.columns.values)

In [ ]:
df.iloc[4][5]